#Normal Pytorch Code
Pytorch Lightning Code follows in the next section

Article -> https://www.learnopencv.com/getting-started-with-pytorch-lightning/

In [2]:
!pip install torch
!pip install torchvision

     |████████████████████████████████| 753.2 MB 6.4 MB/s eta 0:00:015  |                                | 655 kB 236 kB/s eta 0:53:04     |                                | 880 kB 236 kB/s eta 0:53:03     |                                | 1.9 MB 236 kB/s eta 0:52:58     |▏                               | 3.9 MB 10.0 MB/s eta 0:01:15     |▎                               | 5.3 MB 10.0 MB/s eta 0:01:15     |▌                               | 10.6 MB 5.4 MB/s eta 0:02:17     |▊                               | 17.2 MB 3.5 MB/s eta 0:03:28     |▊                               | 17.4 MB 3.5 MB/s eta 0:03:28     |▉                               | 19.3 MB 3.5 MB/s eta 0:03:27     |▉                               | 20.8 MB 2.7 MB/s eta 0:04:34     |█                               | 24.1 MB 2.7 MB/s eta 0:04:32     |█                               | 24.4 MB 4.7 MB/s eta 0:02:36     |█                               | 24.7 MB 4.7 MB/s eta 0:02:36     |█                               | 25.9 MB 4.7 

In [1]:
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch
import torchvision
import os

#setting up configurations
n_epochs = 1
batch_size_train = 128
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5

#initial setup
random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

#loading training dataset
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST(os.getcwd(), train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

#loading testing dataset
test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST(os.getcwd(), train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

#defining the Network

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1,28,kernel_size=5),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2))
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(28,10,kernel_size=2),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2))
        self.dropout1=torch.nn.Dropout(0.25)
        self.fc1=torch.nn.Linear(250,18)
        self.dropout2=torch.nn.Dropout(0.08)
        self.fc2=torch.nn.Linear(18,10)

    def forward(self, x):
        x=self.layer1(x)
        x=self.layer2(x)
        x=self.dropout1(x)
        x=torch.relu(self.fc1(x.view(x.size(0), -1)))
        x=F.leaky_relu(self.dropout2(x))
        
        return F.softmax(self.fc2(x))
#creating a net object    
network = Net()
# setting up the optimiser
optimizer = optim.SGD(network.parameters(), lr=learning_rate,momentum=momentum)

ModuleNotFoundError: No module named 'torch'

In [ ]:
def train(epoch):
    network.train()
    numCorrect=0
    tot=0
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)
        # print(output.shape)
        tot+=len(target)
        numCorrect+=output.argmax(dim=1).eq(target).sum().item()
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
    print("Number of Correctly identified Training Images {} from aset of {}. \nAccuracy= {}".format(numCorrect,tot,numCorrect/tot))
    

In [ ]:
for epoch in range(1, 5 + 1):
    print("At epoch{}".format(epoch))
    train(epoch)

At epoch1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Number of Correctly identified Training Images 6570 from aset of 60000. 
Accuracy= 0.1095 %
At epoch2
Number of Correctly identified Training Images 10254 from aset of 60000. 
Accuracy= 0.1709 %
At epoch3
Number of Correctly identified Training Images 28283 from aset of 60000. 
Accuracy= 0.4713833333333333 %
At epoch4
Number of Correctly identified Training Images 39397 from aset of 60000. 
Accuracy= 0.6566166666666666 %
At epoch5
Number of Correctly identified Training Images 43658 from aset of 60000. 
Accuracy= 0.7276333333333334 %


#Pytorch Lightning Code

In [ ]:
!pip install pytorch_lightning

     |████████████████████████████████| 235kB 6.2MB/s 
     |████████████████████████████████| 829kB 14.1MB/s 
     |████████████████████████████████| 71kB 9.4MB/s 
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=77612728082dba6c4546b2359bee4f8b6d4cc8d5437b3cd49880de5c035ef41e
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
  Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0
  Found existing installation: tqdm 4.38.0
    Uninstalling tqdm-4.38.0:
      Successfully uninstalled tqdm-4.38.0


In [ ]:
#importing necessary libraries
import os
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader,random_split
from torchvision.datasets import MNIST
from torchvision import transforms
import pytorch_lightning as pl
import numpy as np

#defining the model
class smallAndSmartModel(pl.LightningModule):

    def __init__(self):
        #this is the init function where we will defnine the architecture
        super(smallAndSmartModel, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1,28,kernel_size=5),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2))
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(28,10,kernel_size=2),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2))
        self.dropout1=torch.nn.Dropout(0.25)
        self.fc1=torch.nn.Linear(250,18)
        self.dropout2=torch.nn.Dropout(0.08)
        self.fc2=torch.nn.Linear(18,10)
        #we are also defing some variable for counting purposes
        self.valTotal=0
        self.valCorrect=0
        self.trainTotal=0
        self.trainCorrect=0


    def prepare_data(self):
        
        # This contains the manupulation on data that needs to be done only once such as downloading it

        #download the MNIST dataset
        MNIST(os.getcwd(), train=True, download =True)
        MNIST(os.getcwd(), train=False, download =True)
        
        # See here I have set download to false as it is already downloaded
        mnist_train=MNIST(os.getcwd(), train=True, download =False,transform=transforms.ToTensor())

        #dividing into validation and training set
        self.train_set, self.val_set= random_split(mnist_train,[55000,5000])
    
    def train_dataloader(self):
        # REQUIRED
        # This is an essential function. Needs to be included in the code
               
        return DataLoader(self.train_set,batch_size=128,num_workers=32)
        
    def val_dataloader(self):
        # OPTIONAL
        #loading validation dataset
        return DataLoader(self.val_set, batch_size=128,num_workers=32)

    def test_dataloader(self):
        # OPTIONAL
        # loading test dataset
        return DataLoader(MNIST(os.getcwd(), train=False, download=False, transform=transforms.ToTensor()), batch_size=128,num_workers=32)

    def forward(self,x):
        #evaluating the batch data as it moves forward in the netowrk
        x=self.layer1(x)
        x=self.layer2(x)
        x=self.dropout1(x)
        x=torch.relu(self.fc1(x.view(x.size(0), -1)))
        x=F.leaky_relu(self.dropout2(x))

        return F.softmax(self.fc2(x))

    def training_step(self,batch,batch_idx):
        #REQUIRED

        #extracting input and output from the batch
        x,labels=batch

        #doing a forward pass
        pred=self.forward(x)

        self.trainCorrect+=pred.argmax(dim=1).eq(labels).sum().item()
        self.trainTotal+=len(labels)
        #calculating the loss
        train_loss = F.cross_entropy(pred, labels)

        #logs
        logs={"train_loss": train_loss}

        output={
            #REQUIRED: It ie required for us to return "loss"
            "loss": train_loss,
            #optional for logging purposes
            "log": logs
        }

        return output

    def on_epoch_start(self):
        # OPTIONAL
        # runs on the begining of an epoch
        self.valTotal=0
        self.valCorrect=0
        self.testTotal=0
        self.testCorrect=0
        self.trainTotal=0
        self.trainCorrect=0

    def validation_step(self, batch, batch_nb):
        # OPTIONAL
        # The code that runs as we forward pass a validation batch
        x, y = batch
        y_hat = self(x)
        self.valCorrect+=y_hat.argmax(dim=1).eq(y).sum().item()
        self.valTotal+=len(y)
        return {'val_loss': F.cross_entropy(y_hat, y)}

    def validation_epoch_end(self, outputs):
        # OPTIONAL
        # The code that runs as a validation epoch finished
        # Used for metric evaluation
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        print("Number of Correctly identified Validation Images {} from aset of {}. \nAccuracy= {} ".format(self.valCorrect,self.valTotal,self.valCorrect/self.valTotal))
        
        tensorboard_logs = {'val_loss': avg_loss,"Accuracy": self.valCorrect/self.valTotal}
        return {'val_loss': avg_loss, 'log': tensorboard_logs}

    def test_step(self, batch, batch_nb):
        # OPTIONAL
        # The code that runs as we forward pass a test batch

        x, y = batch
        y_hat = self(x)
        self.testCorrect+=y_hat.argmax(dim=1).eq(y).sum().item()
        self.testTotal+=len(y)
        return {'test_loss': F.cross_entropy(y_hat, y)}


    def training_epoch_end(self,outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        tensorboard_logs = {'loss': avg_loss,"Accuracy": self.trainCorrect/self.trainTotal}
        print("Number of Correctly identified Training Set Images {} from a set of {}. \nAccuracy= {} ".format(self.trainCorrect,self.trainTotal,self.trainCorrect/self.trainTotal))
        return {'loss': avg_loss, 'log': tensorboard_logs}

   
    def test_epoch_end(self, outputs):
        # OPTIONAL
        # The code that runs as a validation epoch finished
        # Used for metric evaluation
        print("Number of Correctly identified Testing Images {} from aset of {}. \nAccuracy= {} ".format(self.testCorrect,self.testTotal,self.testCorrect/self.testTotal))

        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        logs = {'test_loss': avg_loss}
        return {'test_loss': avg_loss, 'log': logs, 'progress_bar': logs}

    def configure_optimizers(self):
        # REQUIRED
        # Can return multiple optimizers and learning_rate schedulers
        return torch.optim.Adam(self.parameters(), lr=0.01)


In [ ]:
# Using the Lightning trainer and specifing the requied parameters as arguments
myTrainer=pl.Trainer(gpus=1,max_nb_epochs=10,checkpoint_callback=False)

myModel=smallAndSmartModel()
myTrainer.fit(myModel)

INFO:lightning:GPU available: True, used: True
INFO:lightning:CUDA_VISIBLE_DEVICES: [0]
INFO:lightning:
   | Name     | Type       | Params
------------------------------------
0  | layer1   | Sequential | 728   
1  | layer1.0 | Conv2d     | 728   
2  | layer1.1 | ReLU       | 0     
3  | layer1.2 | MaxPool2d  | 0     
4  | layer2   | Sequential | 1 K   
5  | layer2.0 | Conv2d     | 1 K   
6  | layer2.1 | ReLU       | 0     
7  | layer2.2 | MaxPool2d  | 0     
8  | dropout1 | Dropout    | 0     
9  | fc1      | Linear     | 4 K   
10 | dropout2 | Dropout    | 0     
11 | fc2      | Linear     | 190   


55000


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:66: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Number of Correctly identified Validation Images 67 from aset of 640. 
Accuracy= 0.1046875 %


Number of Correctly identified Validation Images 4701 from aset of 5000. 
Accuracy= 0.9402 %
Number of Correctly identified Training Set Images 46747 from a set of 55000. 
Accuracy= 0.8499454545454546 %


...

Number of Correctly identified Validation Images 4708 from aset of 5000. 
Accuracy= 0.9416 %
Number of Correctly identified Training Set Images 48794 from a set of 55000. 
Accuracy= 0.8871636363636364 %


Number of Correctly identified Validation Images 4588 from aset of 5000. 
Accuracy= 0.9176 %
Number of Correctly identified Training Set Images 48824 from a set of 55000. 
Accuracy= 0.8877090909090909 %


Number of Correctly identified Validation Images 4002 from aset of 5000. 
Accuracy= 0.8004 %
Number of Correctly identified Training Set Images 48466 from a set of 55000. 
Accuracy= 0.8812 %


Number of Correctly identified Validation Images 4624 from aset of 5000. 
Accuracy= 0.9248 %
Number of Correctly identified Training Set Images 47588 from a set of 55000. 
Accuracy= 0.8652363636363636 %



1

In [ ]:
myTrainer.test()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:66: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Number of Correctly identified Testing Images 9305 from aset of 10000. 
Accuracy= 0.9305 %
--------------------------------------------------------------------------------
TEST RESULTS
{'test_loss': tensor(1.5299, device='cuda:0')}
--------------------------------------------------------------------------------

